# Introduction
Building on the model visualization of Mol et al., the acetone production pathway is added 

In [1]:
import cameo
import pandas as pd
import cobra.io
import escher
from escher import Builder
from cobra import Reaction, Metabolite
from os.path import dirname, join, pardir

data_dir = "/Users/phivri/Documents/GitHub/Biosustain/MoGeoModel"

In [6]:
core = cobra.io.read_sbml_model(join(data_dir,'Models/e_coli_core.xml'))


solution = core.optimize()
Builder(model = core, map_name='e_coli_core.Core metabolism', reaction_data = solution.fluxes.to_dict())

Builder(reaction_data={'PFK': 7.477381962160287, 'PFL': 0.0, 'PGI': 4.860861146496827, 'PGK': -16.023526143167…

In [2]:
model = cobra.io.read_sbml_model(join(data_dir,'Models/p-thermo_acetone_anaerobic.xml'))

medium = model.medium
medium["EX_o2_e"] = 1000.0
medium["EX_ac_e"] = 10.0
medium["EX_glc__D_e"] = 0.0
model.medium = medium

solution = model.optimize()
Builder(model = model, map_json = join(data_dir,'Models/p-thermo_acetone_map.json'), reaction_data = solution.fluxes.to_dict())

Builder(reaction_data={'IDPh': 10.436720375457611, 'CAT': 0.0, 'PDHam1hi': 0.0, 'HYDA': 0.0, 'MALHYDRO': 0.0, …

In [3]:
model.reactions.get_by_id("EX_Biomass_e")

Reaction identifier,EX_Biomass_e
Name,Biomass exchange
Memory address,0x07fe3d54c2550
Stoichiometry,Biomass_e --> Biomass -->
GPR,
Lower bound,0
Upper bound,1000.0


In [4]:
#Desired ratio of max growth, chosen from PPP:
ratio = 0.5

XFlux = model.slim_optimize()*ratio

with model:
    model.reactions.get_by_id("EX_Biomass_e").bounds = XFlux, XFlux
    model.objective = "EX_act_e"
    PFlux = model.slim_optimize()

set_flux = model.problem.Constraint(
    XFlux * model.reactions.EX_act_e.flux_expression - PFlux * model.reactions.EX_Biomass_e.flux_expression,
    lb=0,
    ub=0)
model.add_cons_vars(set_flux)

solution = model.optimize()

Builder(model = model, map_json = join(data_dir,'Models/AcetonePathway.json'), reaction_data = solution.fluxes.to_dict())

Builder(reaction_data={'IDPh': 8.237433559589475, 'CAT': -5.141076771694051e-09, 'PDHam1hi': 0.0, 'HYDA': 0.0,…